## Hotfix passim-no-bp

When dumping the json to files, there was a problem, this notebook aims to hotfix it

### Imports

In [1]:
import os
import json
import boto3
import jsonlines
from smart_open import open as s_open
import dask.bag as db
from impresso_commons.utils.s3 import IMPRESSO_STORAGEOPT, fixed_s3fs_glob, get_s3_client, read_jsonlines, alternative_read_text
from impresso_commons.path.path_s3 import list_files, fetch_files
import pandas as pd
import ast

### Code

In [16]:
s3_bucket = '30-passim-rebuilt-sandbox/passim-no-bp'

no_bp_files = fixed_s3fs_glob(os.path.join(s3_bucket, '*.jsonl.bz2'))
no_bp_files[:5]

['s3://30-passim-rebuilt-sandbox/passim-no-bp/BDC-0001.jsonl.bz2',
 's3://30-passim-rebuilt-sandbox/passim-no-bp/avenirgdl-0001.jsonl.bz2',
 's3://30-passim-rebuilt-sandbox/passim-no-bp/avenirgdl-0002.jsonl.bz2',
 's3://30-passim-rebuilt-sandbox/passim-no-bp/avenirgdl-0003.jsonl.bz2',
 's3://30-passim-rebuilt-sandbox/passim-no-bp/avenirgdl-0004.jsonl.bz2']

In [19]:
text1 = alternative_read_text(no_bp_files[0], IMPRESSO_STORAGEOPT)
text1[:5]

['{"series": "BDC", "date": "1839-01-20", "id": "BDC-1839-01-20-a-i0001", "cc": true, "lg": "fr", "pages": [{"id": "BDC-1839-01-20-a-p0001", "seq": 1, "regions": [{"start": 0, "length": 5, "coords": {"x": 174, "y": 502, "w": 142, "h": 41}}, {"start": 5, "length": 5, "coords": {"x": 81, "y": 565, "w": 50, "h": 23}}, {"start": 10, "length": 3, "coords": {"x": 144, "y": 565, "w": 18, "h": 23}}, {"start": 13, "length": 8, "coords": {"x": 175, "y": 565, "w": 71, "h": 23}}, {"start": 21, "length": 7, "coords": {"x": 259, "y": 565, "w": 55, "h": 23}}, {"start": 28, "length": 6, "coords": {"x": 328, "y": 565, "w": 43, "h": 23}}, {"start": 34, "length": 3, "coords": {"x": 384, "y": 565, "w": 23, "h": 23}}, {"start": 37, "length": 5, "coords": {"x": 423, "y": 565, "w": 44, "h": 23}}, {"start": 42, "length": 3, "coords": {"x": 32, "y": 594, "w": 18, "h": 23}}, {"start": 45, "length": 3, "coords": {"x": 64, "y": 594, "w": 22, "h": 23}}, {"start": 48, "length": 5, "coords": {"x": 99, "y": 594, "w":

In [24]:
text_bag = db.read_text(no_bp_files[10], storage_options=IMPRESSO_STORAGEOPT).map(json.load)
text2 = text_bag.compute()
text2[:5]

AttributeError: 'str' object has no attribute 'read'

In [20]:
session = boto3.Session(
    aws_access_key_id=IMPRESSO_STORAGEOPT["key"],
    aws_secret_access_key=IMPRESSO_STORAGEOPT["secret"],
)
s3_endpoint = IMPRESSO_STORAGEOPT["client_kwargs"]["endpoint_url"]
transport_params = {
    "client": session.client("s3", endpoint_url=s3_endpoint),
}

In [23]:
with s_open(no_bp_files[0], "r", encoding='utf-8', transport_params=transport_params) as infile:
    dataread = infile.readlines()
        
data = json.loads(dataread[2])
data

{'series': 'BDC',
 'date': '1839-01-20',
 'id': 'BDC-1839-01-20-a-i0003',
 'cc': True,
 'lg': 'fr',
 'pages': [{'id': 'BDC-1839-01-20-a-p0003',
   'seq': 3,
   'regions': [{'start': 0,
     'length': 11,
     'coords': {'x': 330, 'y': 54, 'w': 281, 'h': 35}},
    {'start': 11,
     'length': 3,
     'coords': {'x': 455, 'y': 100, 'w': 33, 'h': 23}},
    {'start': 14,
     'length': 19,
     'coords': {'x': 29, 'y': 141, 'w': 887, 'h': 57}},
    {'start': 33,
     'length': 3,
     'coords': {'x': 176, 'y': 205, 'w': 40, 'h': 28}},
    {'start': 36,
     'length': 3,
     'coords': {'x': 232, 'y': 205, 'w': 41, 'h': 28}},
    {'start': 39,
     'length': 13,
     'coords': {'x': 288, 'y': 205, 'w': 249, 'h': 28}},
    {'start': 52,
     'length': 11,
     'coords': {'x': 558, 'y': 205, 'w': 207, 'h': 28}},
    {'start': 63,
     'length': 2,
     'coords': {'x': 30, 'y': 245, 'w': 20, 'h': 23}},
    {'start': 65, 'length': 2, 'coords': {'x': 50, 'y': 245, 'w': 8, 'h': 23}},
    {'start'

In [24]:
json_data = json.dump(text2[0])
json_without_slash = json.loads(json_data)
json_without_slash

TypeError: dump() missing 1 required positional argument: 'fp'

## Test to see what dumps things correclty

In [9]:
bp_s3_path='s3://40-processed-data-sandbox/text-reuse/text-reuse_v1-0-0/boilerplate/bp.pkl'
output_bucket='s3://30-passim-rebuilt-sandbox/passim-no-bp/' 
input_bucket='s3://30-passim-rebuilt-sandbox/passim/'

In [10]:
bp_df = pd.read_pickle(bp_s3_path, storage_options=IMPRESSO_STORAGEOPT).persist()
np = 'BDC'
#existing_files = fixed_s3fs_glob(f'{os.path.join(output_bucket, np)}*.bz2')
passim_rebuilt_files = fixed_s3fs_glob(f'{os.path.join(input_bucket, np)}/*.bz2')
output_files = [
        f'{os.path.join(output_bucket, np)}-{str(n+1).zfill(4)}.jsonl.bz2' for n, f in enumerate(passim_rebuilt_files)
    ]
n_partitions = len(passim_rebuilt_files)

In [11]:
np_bp_df = bp_df[bp_df.id.str.contains(np)].set_index('id').compute()

In [12]:
passim_data_df = (
        db.read_text(passim_rebuilt_files, storage_options=IMPRESSO_STORAGEOPT)
        .map(json.loads)
        .map(lambda d: {'id': d['id'], 'document': d})
        .to_dataframe()
        .set_index('id')
        .persist()
    )

In [13]:
tmp_df = passim_data_df.join(np_bp_df, how='outer')
filtered_df = tmp_df[tmp_df.is_boilerplate.isnull()]

In [15]:
filtered_df.head()

,document,is_boilerplate
id,,
BDC-1839-01-20-a-i0001,"{'series': 'BDC', 'date': '1839-01-20', 'id': ...",NaN
BDC-1839-01-20-a-i0002,"{'series': 'BDC', 'date': '1839-01-20', 'id': ...",NaN
BDC-1839-01-20-a-i0003,"{'series': 'BDC', 'date': '1839-01-20', 'id': ...",NaN
BDC-1839-01-20-a-i0004,"{'series': 'BDC', 'date': '1839-01-20', 'id': ...",NaN
BDC-1839-01-20-a-i0005,"{'series': 'BDC', 'date': '1839-01-20', 'id': ...",NaN


In [14]:
#%%timeit
future = (
    filtered_df.reset_index()
    .to_bag()
    .map(lambda i: ast.literal_eval(i[1]))
    #.persist()
    .map(json.dumps) # TODO warning DOES NOT OUTPUT AS JSON
    .repartition(n_partitions)
    .to_textfiles(output_files, storage_options=IMPRESSO_STORAGEOPT)
)

In [140]:
#%%timeit
future = (
    filtered_df.reset_index()
    .to_bag()
    .map(lambda i: i[1])
    #.map(ast.literal_eval)
    #.map(json.dumps) # TODO warning DOES NOT OUTPUT AS JSON
    .repartition(n_partitions)
    .to_textfiles(output_files, storage_options=IMPRESSO_STORAGEOPT)
)

In [61]:
%%timeit
future = (
    filtered_df.reset_index()
    .to_bag()
    .map(lambda i: i[1])
    #.persist()
    .map(json.dumps) # TODO warning DOES NOT OUTPUT AS JSON
    .repartition(n_partitions)
    .to_textfiles(output_files, storage_options=IMPRESSO_STORAGEOPT)
)

2.28 s ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [141]:
future[0]

'30-passim-rebuilt-sandbox/passim-no-bp/BDC-0001.jsonl.bz2'

In [56]:
future = (
    filtered_df.reset_index()
    .to_bag()
    .map(lambda i: ast.literal_eval(i[1]))
)
future.take(1)

({'series': 'BDC',
  'date': '1839-01-20',
  'id': 'BDC-1839-01-20-a-i0001',
  'cc': True,
  'lg': 'fr',
  'pages': [{'id': 'BDC-1839-01-20-a-p0001',
    'seq': 1,
    'regions': [{'start': 0,
      'length': 5,
      'coords': {'x': 174, 'y': 502, 'w': 142, 'h': 41}},
     {'start': 5,
      'length': 5,
      'coords': {'x': 81, 'y': 565, 'w': 50, 'h': 23}},
     {'start': 10,
      'length': 3,
      'coords': {'x': 144, 'y': 565, 'w': 18, 'h': 23}},
     {'start': 13,
      'length': 8,
      'coords': {'x': 175, 'y': 565, 'w': 71, 'h': 23}},
     {'start': 21,
      'length': 7,
      'coords': {'x': 259, 'y': 565, 'w': 55, 'h': 23}},
     {'start': 28,
      'length': 6,
      'coords': {'x': 328, 'y': 565, 'w': 43, 'h': 23}},
     {'start': 34,
      'length': 3,
      'coords': {'x': 384, 'y': 565, 'w': 23, 'h': 23}},
     {'start': 37,
      'length': 5,
      'coords': {'x': 423, 'y': 565, 'w': 44, 'h': 23}},
     {'start': 42,
      'length': 3,
      'coords': {'x': 32, 'y'

In [46]:
passim_data_df_T = (
        db.read_text(passim_rebuilt_files, storage_options=IMPRESSO_STORAGEOPT)
        .map(json.loads)
        .map(lambda d: {'id': d['id'], 'document': d})
        #.to_dataframe()
        #.set_index('id')
        #.persist()
    ).compute()

In [51]:
filtered_df.head(), filtered_df.describe()

(                                                                 document  \
 id                                                                          
 BDC-1839-01-20-a-i0001  {'series': 'BDC', 'date': '1839-01-20', 'id': ...   
 BDC-1839-01-20-a-i0002  {'series': 'BDC', 'date': '1839-01-20', 'id': ...   
 BDC-1839-01-20-a-i0003  {'series': 'BDC', 'date': '1839-01-20', 'id': ...   
 BDC-1839-01-20-a-i0004  {'series': 'BDC', 'date': '1839-01-20', 'id': ...   
 BDC-1839-01-20-a-i0005  {'series': 'BDC', 'date': '1839-01-20', 'id': ...   
 
                        is_boilerplate  
 id                                     
 BDC-1839-01-20-a-i0001            NaN  
 BDC-1839-01-20-a-i0002            NaN  
 BDC-1839-01-20-a-i0003            NaN  
 BDC-1839-01-20-a-i0004            NaN  
 BDC-1839-01-20-a-i0005            NaN  ,
 Dask DataFrame Structure:
               document is_boilerplate
 npartitions=1                        
                 string         string
                    

## other option, not using dask dataframes

In [25]:
#bp_s3_path='s3://41-processed-data-staging/text-reuse/text-reuse_v1-0-0/boilerplate/bp.pkl'
bp_s3_path='s3://40-processed-data-sandbox/text-reuse/text-reuse_v1-0-0/boilerplate/bp.pkl'
output_bucket='s3://30-passim-rebuilt-sandbox/passim-no-bp/' 
input_bucket='s3://30-passim-rebuilt-sandbox/passim/'

In [26]:
bp_df = pd.read_pickle(bp_s3_path, storage_options=IMPRESSO_STORAGEOPT).persist()
np = 'excelsior'
#existing_files = fixed_s3fs_glob(f'{os.path.join(output_bucket, np)}*.bz2')
passim_rebuilt_files = fixed_s3fs_glob(f'{os.path.join(input_bucket, np)}/*.bz2')
output_files = [
        f'{os.path.join(output_bucket, np)}-{str(n+1).zfill(4)}.jsonl.bz2' for n, f in enumerate(passim_rebuilt_files)
    ]
n_partitions = len(passim_rebuilt_files)

In [39]:
bp_df = pd.read_pickle(bp_s3_path, storage_options=IMPRESSO_STORAGEOPT).drop(columns=["is_boilerplate"]).persist()

In [40]:
bp_df.head()

,id
0,BNN-1887-06-01-a-i0016
1,BNN-1887-05-28-a-i0021
2,BNN-1888-01-08-a-i0012
3,BNN-1887-12-30-a-i0012
4,CDV-1844-04-06-a-i0001


In [35]:
np_bp_df = set(bp_df[bp_df.id.str.contains(np)].to_bag().map(lambda x: x[0]).compute())

In [36]:
np_bp_df

{'excelsior-1910-12-10-a-i0082',
 'excelsior-1910-12-10-a-i0157',
 'excelsior-1910-12-14-a-i0126',
 'excelsior-1910-12-15-a-i0127',
 'excelsior-1910-12-16-a-i0117',
 'excelsior-1910-12-17-a-i0247',
 'excelsior-1911-03-17-a-i0151',
 'excelsior-1911-03-24-a-i0153',
 'excelsior-1911-04-29-a-i0043',
 'excelsior-1911-04-30-a-i0038',
 'excelsior-1911-08-30-a-i0110',
 'excelsior-1911-08-31-a-i0100',
 'excelsior-1911-12-27-a-i0191',
 'excelsior-1912-01-03-a-i0152',
 'excelsior-1912-02-04-a-i0205',
 'excelsior-1912-02-11-a-i0218',
 'excelsior-1912-03-07-a-i0210',
 'excelsior-1912-03-14-a-i0178',
 'excelsior-1912-07-05-a-i0224',
 'excelsior-1912-07-12-a-i0224',
 'excelsior-1912-09-03-a-i0176',
 'excelsior-1912-09-10-a-i0146',
 'excelsior-1912-09-15-a-i0113',
 'excelsior-1912-09-16-a-i0103',
 'excelsior-1912-10-22-a-i0154',
 'excelsior-1912-10-24-a-i0183',
 'excelsior-1912-12-06-a-i0125',
 'excelsior-1912-12-07-a-i0111',
 'excelsior-1912-12-22-a-i0230',
 'excelsior-1912-12-31-a-i0195',
 'excelsio

In [47]:
%%timeit
bp_df = pd.read_pickle(bp_s3_path, storage_options=IMPRESSO_STORAGEOPT).persist()
np_bp_df = set(bp_df[bp_df.id.str.contains(np)].to_bag().map(lambda x: x[0]).compute())

1.02 s ± 41.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%%timeit
bp_df_1 = pd.read_pickle(bp_s3_path, storage_options=IMPRESSO_STORAGEOPT).persist()
np_bp_df_1 = set(bp_df_1[bp_df_1.id.str.contains(np)].to_bag().map_partitions(lambda x: [y[0] for y in x]).compute())

1.02 s ± 20.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
%%timeit
bp_df_2 = pd.read_pickle(bp_s3_path, storage_options=IMPRESSO_STORAGEOPT).drop(columns=["is_boilerplate"]).persist()
np_bp_df_2 = set(bp_df_2[bp_df_2.id.str.contains(np)].to_bag().map_partitions(lambda x: [y[0] for y in x]).compute())

1.02 s ± 32.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
np_bp_df_2 == np_bp_df

True

In [38]:
passim_data_df = db.read_text(passim_rebuilt_files, storage_options=IMPRESSO_STORAGEOPT).map(json.loads).persist()

passim_filtered = passim_data_df.filter(lambda d: d['id'] not in np_bp_df)

KeyboardInterrupt: 

In [ ]:
passim_filtered.take(1)

In [105]:
passim_filtered.take(1)

({'series': 'excelsior',
  'date': '1910-12-27',
  'id': 'excelsior-1910-12-27-a-i0001',
  'cc': True,
  'lg': None,
  'pages': [{'id': 'excelsior-1910-12-27-a-p0001',
    'seq': 1,
    'regions': [{'start': 0,
      'length': 3,
      'coords': {'x': 1405, 'y': 4149, 'w': 98, 'h': 44}},
     {'start': 3,
      'length': 13,
      'coords': {'x': 1549, 'y': 4142, 'w': 492, 'h': 50}},
     {'start': 16,
      'length': 11,
      'coords': {'x': 2087, 'y': 4144, 'w': 430, 'h': 51}},
     {'start': 27,
      'length': 3,
      'coords': {'x': 2559, 'y': 4143, 'w': 90, 'h': 44}},
     {'start': 30,
      'length': 2,
      'coords': {'x': 2695, 'y': 4139, 'w': 24, 'h': 16}},
     {'start': 32,
      'length': 5,
      'coords': {'x': 2763, 'y': 4142, 'w': 164, 'h': 44}},
     {'start': 37,
      'length': 5,
      'coords': {'x': 2972, 'y': 4140, 'w': 186, 'h': 45}},
     {'start': 42,
      'length': 2,
      'coords': {'x': 3189, 'y': 4139, 'w': 24, 'h': 15}},
     {'start': 44,
      'l

In [106]:
future = (
    passim_filtered
        .sort
        .map(json.dumps)
        .repartition(n_partitions)
        .to_textfiles(output_files, storage_options=IMPRESSO_STORAGEOPT)
    )

In [148]:
old = 20+13029+6622+5108
new = 4183+8866+6622+5108

old, new

(24779, 24779)